<a href="https://colab.research.google.com/github/Blue22796/Auto-searcher/blob/main/mount_onedrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown Tutorial adapted from https://www.youtube.com/watch?v=U6YPgARhRzA

#@markdown The first step is to download a copy of rclone from https://downloads.rclone.org **in your system**, unzip it and execute in a terminal `rclone authorize "onedrive"`, log into your account and copy the access token. Then run the cell to install `rclone` in Colab.

!wget https://downloads.rclone.org/v1.56.0/rclone-v1.56.0-linux-amd64.deb
!apt install ./rclone-v1.56.0-linux-amd64.deb

In [ ]:
#@markdown Run this cell, and in rclone config, select `n` for new remote, name it `onedrive` and check the number in the list corresponding to onedrive (`26` in the current version). Then press enter for `client_id` and for `client_secret`, then `n` to avoid avanced config and `n` for auto config.
#@markdown Then paste the access token code generated in your machine, and select number 1 for "Onedrive personal" or "Onedrive business"
#@markdown Then, in "found drives" check that the configuration is ok, depending on the one you want to mount. Then select `y` twice and `q` to quit the configuration.
!rclone config

In [ ]:
#@markdown Then, you can mount the drive in `\content\gdrive` to use it seamlessly in the ZeroCostDL4Mic notebooks
import os
import sys
folder_name = "gdrive" #@param {type:"string"}
folder=str('/content/'+folder_name)
print("Mounted in",folder)
os.mkdir(folder)
os.system("nohup rclone --vfs-cache-mode writes mount onedrive: %s &" % folder)

In [25]:
!python -m pip install --upgrade pip setuptools wheel

!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-libtorrent is already the newest version (2.0.5-5).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [29]:
!python -m pip install --upgrade pip setuptools wheel

!python -m pip install lbry-libtorrent
!apt install python3-libtorrent
import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-libtorrent is already the newest version (2.0.5-5).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [30]:
params = {"save_path": "/content/sample_data/Torrent"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


Enter Magnet Link Or Type Exit: magnet:?xt=urn:btih:64FA662F6FCADD59BAC238715F8E45BC85D6C335&dn=Monster+Hunter+World%3A+Iceborne+-+Master+Edition+%28v15.11.01%2F421471+%2B+242+DLCs%2C+MULTi13%29+%5BFitGirl+Repack%5D&tr=udp%3A%2F%2F46.148.18.250%3A2710&tr=udp%3A%2F%2Fopentor.org%3A2710&tr=udp%3A%2F%2Ftracker.coppersurfer.tk%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=http%3A%2F%2Ftracker.dler.org%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.leechers-paradise.org%3A6969%2Fannounce&tr=udp%3A%2F%2F9.rarbg.me%3A2730%2Fannounce&tr=udp%3A%2F%2F9.rarbg.to%3A2770%2Fannounce&tr=udp%3A%2F%2Ftracker.pirateparty.gr%3A6969%2Fannounce&tr=http%3A%2F%2Fretracker.local%2Fannounce&tr=http%3A%2F%2Fretracker.ip.ncnet.ru%2Fannounce&tr=udp%3A%2F%2Fexodus.desync.com%3A6969%2Fannounce&tr=udp%3A%2F%2Fipv4.tracker.harry.lu%3A80%2Fannounce&tr=udp%3A%2F%2Fopen.stealth.si%3A80%2Fannounce&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=http%3A%2F%2Ftracker.openbittorrent.com%3

In [ ]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)


FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…